In [1]:
!pip install selenium

In [2]:
import pandas as pd
import os
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By

In [3]:
CHROMEDRIVER_PATH = './chromedriver'
WINDOW_SIZE = "1920,1080"

In [4]:
# Comman function for init driver
def init_driver(url):
    driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH)
    driver.maximize_window()
    driver.get(url)
    return driver

In [5]:
def wait_for_page_load(xpath_url):
    try:
        element_present = EC.presence_of_element_located((By.XPATH, xpath_url))
        WebDriverWait(driver, 60).until(element_present)
    except TimeoutException:
        print("Timed out waiting for page to load")

# Q1: Write a python program to scrape data for “Data Analyst” Job position in “Bangalore” location. You have to scrape the job-title, job-location, company_name, experience_required. You have to scrape first 10 jobs data.

In [6]:
# init driver
driver = init_driver('https://www.naukri.com/')

In [7]:
# add data to keyword field 
driver.find_element_by_xpath("//form//input[@name='keyword']").send_keys('Data Analyst')

In [8]:
# add data to location filed
driver.find_element_by_xpath("//form//input[@name='location']").send_keys('Bangalore')

In [9]:
# hit search 
driver.find_element_by_xpath("//div[@class='search-btn']/button").click()

In [10]:
# wait for page load
wait_for_page_load('//article[@class="jobTuple bgWhite br4 mb-8"]')    

# finding top 10 boxes 
all_box = driver.find_elements_by_xpath('//article[@class="jobTuple bgWhite br4 mb-8"]')[:10]
    
list_for_links = []
final_data = []

# Ittrating over box to find details
for one_box in all_box:
    job_title = one_box.find_element_by_xpath(".//a").text
    company_name = one_box.find_element_by_xpath(".//div[contains(@class,'companyInfo')]/a").text
    job_location = one_box.find_element_by_xpath(".//ul/li[contains(@class,'location')]").text 
    experience_required = one_box.find_element_by_xpath(".//ul/li[contains(@class,'experience')]").text 
    
    link_of_job = one_box.find_element_by_xpath(".//a").get_attribute('href')
    list_for_links.append(link_of_job)
    
    final_data.append({
        'job_title':job_title,
        'company_name':company_name,
        'job_location':job_location,
        'experience_required':experience_required
    })
    
job_data = pd.DataFrame(final_data)
job_data

,job_title,company_name,job_location,experience_required
0,Senior Data Analyst,Virtusa Consulting Services Pvt Ltd,"Hyderabad/Secunderabad, Pune, Gurgaon/Gurugram...",8-12 Yrs
1,Senior Data Analyst,Flipkart,Bangalore/Bengaluru,2-3 Yrs
2,Senior Data Analyst,Flipkart,Bangalore/Bengaluru,3-8 Yrs
3,SENIOR MARKETING DATA ANALYST,McAfee Software (India) Pvt. Ltd,Bangalore/Bengaluru,3-7 Yrs
4,Data Analyst - Quantitative Support Team,TresVista Financial,"Mumbai, Pune, Bangalore/Bengaluru",0-1 Yrs
5,Urgent Openings For Data Analysts.,Flipkart Internet Private Limited,Bangalore/Bengaluru(Devarabeesana Halli)\n(WFH...,1-6 Yrs
6,Business Data Analyst - Alteryx/Hadoop,Talent Placers,Bangalore/Bengaluru,5-8 Yrs
7,Data Analyst,Flipkart,Bangalore/Bengaluru,1-2 Yrs
8,Senior Data Analyst,Valgen Decision Technologies Pvt Ltd,Bangalore/Bengaluru,5-7 Yrs
9,Senior Data Analyst,Kwalee ltd.,Bangalore/Bengaluru,5-10 Yrs


# 2.Please note that you have to scrape full job description. For that you may have to open each job separately as shown below

In [11]:
# Itterate through each link to find the job description
final_jd_list = []
for one_link in list_for_links:
    driver.get(one_link)
    try:
        # Finding for simple page
        job_description = driver.find_element_by_xpath('//section[@class="job-desc"]').text
        
    except:
        # fiding in diffrent way if got an error 
        job_description = driver.find_element_by_xpath('//div[contains(@itemprop,"description")]').text
    
    job_description = job_description.replace('\n', ' ')
    # appending to list
    final_jd_list.append({'new_page_driver':job_description})
    
desc_df = pd.DataFrame(final_jd_list)
job_data['job_description'] = desc_df
job_data

,job_title,company_name,job_location,experience_required,job_description
0,Senior Data Analyst,Virtusa Consulting Services Pvt Ltd,"Hyderabad/Secunderabad, Pune, Gurgaon/Gurugram...",8-12 Yrs,Job Description Roles and Responsibilities Tot...
1,Senior Data Analyst,Flipkart,Bangalore/Bengaluru,2-3 Yrs,Job description Primary responsibility of the ...
2,Senior Data Analyst,Flipkart,Bangalore/Bengaluru,3-8 Yrs,Job description About the role: Data Analyst i...
3,SENIOR MARKETING DATA ANALYST,McAfee Software (India) Pvt. Ltd,Bangalore/Bengaluru,3-7 Yrs,Job description Role Overview: This position p...
4,Data Analyst - Quantitative Support Team,TresVista Financial,"Mumbai, Pune, Bangalore/Bengaluru",0-1 Yrs,Job description Data Analyst - Quantitative Su...
5,Urgent Openings For Data Analysts.,Flipkart Internet Private Limited,Bangalore/Bengaluru(Devarabeesana Halli)\n(WFH...,1-6 Yrs,Job description We are looking for Data Analys...
6,Business Data Analyst - Alteryx/Hadoop,Talent Placers,Bangalore/Bengaluru,5-8 Yrs,Job description Our client is having a require...
7,Data Analyst,Flipkart,Bangalore/Bengaluru,1-2 Yrs,Job description About the role: Data Analyst i...
8,Senior Data Analyst,Valgen Decision Technologies Pvt Ltd,Bangalore/Bengaluru,5-7 Yrs,Job description The Senior Data Analyst shall ...
9,Senior Data Analyst,Kwalee ltd.,Bangalore/Bengaluru,5-10 Yrs,Job description Kwalee is one of the world's l...


# Q3: In this question you have to scrape data using the filters available on the webpage as shown below:

* The location filter to be used is “Delhi/NCR”
* The salary filter to be used is “3-6” lakhs

In [18]:
# init driver
driver.get('https://www.naukri.com/')

In [19]:
# add data to keyword field 
driver.find_element_by_xpath("//form//input[@name='keyword']").send_keys('Data Analyst')

In [20]:
# add data to location filed
driver.find_element_by_xpath("//form//input[@name='location']").send_keys('Bangalore')

In [21]:
# hit search 
driver.find_element_by_xpath("//div[@class='search-btn']/button").click()

In [22]:
# wait for page load
wait_for_page_load("//input[@id='chk-Delhi / NCR-cityTypeGid-']")    


# Appling filters on the page

# appling location filter
driver.find_element_by_xpath("//span[@title='Delhi / NCR']").click()


# wait for page load
wait_for_page_load("//span[@title='3-6 Lakhs']")    


# appling salary filter
driver.find_element_by_xpath("//span[@title='3-6 Lakhs']").click()


# wait for page load
wait_for_page_load('//article[@class="jobTuple bgWhite br4 mb-8"]') 

# finding top 10 boxes 
all_box = driver.find_elements_by_xpath('//article[@class="jobTuple bgWhite br4 mb-8"]')[:10]
    
list_for_links = []
final_data = []

# Ittrating over box to find details
for one_box in all_box:
    job_title = one_box.find_element_by_xpath(".//a").text
    company_name = one_box.find_element_by_xpath(".//div[contains(@class,'companyInfo')]/a").text
    job_location = one_box.find_element_by_xpath(".//ul/li[contains(@class,'location')]").text 
    experience_required = one_box.find_element_by_xpath(".//ul/li[contains(@class,'experience')]").text 
    
    link_of_job = one_box.find_element_by_xpath(".//a").get_attribute('href')
    list_for_links.append(link_of_job)
    
    final_data.append({
        'job_title':job_title,
        'company_name':company_name,
        'job_location':job_location,
        'experience_required':experience_required
    })
    
job_data = pd.DataFrame(final_data)
job_data

,job_title,company_name,job_location,experience_required
0,Data Analyst/Data Scientist,Skillenable Fintech Private Limited,"Bangalore/Bengaluru, Delhi / NCR, Mumbai (All ...",2-5 Yrs
1,"EY GDS hiring For Data Analyst(SQL,Python acro...",EYGBS,"Kolkata, Hyderabad/Secunderabad, Pune, Chennai...",4-9 Yrs
2,Data Analyst,Podium Systems,"Pune, Gurgaon/Gurugram, Chennai, Bangalore/Ben...",6-11 Yrs
3,Data Analyst,Metaphor Infotech,"Gurgaon/Gurugram, Bangalore/Bengaluru\n(WFH du...",2-6 Yrs
4,Scouting For Data Analyst,Metaphor Infotech,"Gurgaon/Gurugram, Bangalore/Bengaluru",5-10 Yrs
5,Data Analyst,TMD - The Modern Dimension,"New Delhi, Bangalore/Bengaluru",3-8 Yrs
6,Senior Data Analyst,UrbanClap,"Gurgaon/Gurugram, Bangalore/Bengaluru",2-4 Yrs
7,Tcs Hiring For Data Quality Analyst,Tata Consultancy Services Ltd.,"Kolkata, Hyderabad/Secunderabad, Pune, Chennai...",4-9 Yrs
8,Hiring Data Analyst For Leading E commerce GIANT,Rivera Manpower Services,"Pune, Bangalore/Bengaluru, Delhi / NCR",1-5 Yrs
9,Opening For Data Analyst 2-6 Yrs Exp ( 3rd Par...,Sellcraft Global Solutions Private Limited,"Kolkata, Hyderabad/Secunderabad, Pune, Ahmedab...",2-6 Yrs
